# Infer CYTOSOL - part 3

--------------

## OBJECTIVE:  Infer sub-cellular component #3: CYTOSOL  in order to understand interactome 

To measure shape, position, and size of the cytosol of the cell body -- the soma minus the nucleus.    

Dependencies:
The CYTOSOL inference rely on the NUCLEI AND SOMA inference.  Therefore all of the sub-cellular objects rely on this segmentation.





# IMPORTS

In [5]:

# this needs to be organzied to explain the imports
from pathlib import Path
import os
from collections import defaultdict

import numpy as np
import scipy
from scipy import ndimage as ndi

import napari

# function for core algorithm
import aicssegmentation
from aicssegmentation.core.seg_dot import dot_3d_wrapper, dot_slice_by_slice, dot_2d_slice_by_slice_wrapper, dot_3d
from aicssegmentation.core.pre_processing_utils import ( intensity_normalization, 
                                                         image_smoothing_gaussian_3d,  
                                                         image_smoothing_gaussian_slice_by_slice )
from aicssegmentation.core.utils import topology_preserving_thinning, hole_filling
from aicssegmentation.core.MO_threshold import MO
from aicssegmentation.core.vessel import filament_2d_wrapper, vesselnessSliceBySlice
from aicssegmentation.core.output_utils import   save_segmentation,  generate_segmentation_contour
                                                 
from skimage import filters, img_as_float
from skimage import morphology

from skimage.segmentation import watershed
from skimage.feature import peak_local_max
from skimage.morphology import remove_small_objects, binary_closing, ball , dilation   # function for post-processing (size filter)
from skimage.measure import label
# # package for io 
# from aicsimageio import AICSImage

from napari.utils.notebook_display import nbscreenshot



%load_ext autoreload
%autoreload 2
#import .infer_subc.base
from infer_subc.base import *

viewer = None

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# IMAGE PROCESSING Objective 3:  infer CYTOSOL
  [OUTLINE: Objective #2](#summary-of-objectives)
## summary of steps

INPUT
- labeled NUCLEI (objective #1)
- labeled SOMA (objective #2)

PRE-PROCESSING
- erode Nuclei (shrink)
- CORE-PROCESSIN
  - XOR(NUCLEI, SOMA)

- POST-PROCESSING
  - N/A

OUTPUT
- mask of CYTOSOL
- CYTOSOL object (intensity)



# INPUT

In [8]:
###################
# INPUT
###################
##  set up files

data_path = Path( f"{os.getenv('HOME')}/Projects/Imaging/mcz_subcell/data")

out_path = data_path / "inferred_objects" 
object_name = 'NU_object'
NU_object, meta_dict_t = read_input_image( out_path/ f"{object_name}.ome.tiff" )
NU_labels = label(NU_object)


object_name = 'SO_object3'
label_name = 'SO_label3'


SO_object, meta_dict_t = read_input_image( out_path/ f"{object_name}.ome.tiff" )
SO_labels, meta_dict_t = read_input_image( out_path/ f"{label_name}.ome.tiff" )



'/Users/ahenrie/Projects/Imaging/mcz_subcell/data/inferred_objects/SO_object3.ome.tiff'

/opt/anaconda3/envs/napariNEW/lib/python3.9/site-packages/ome_types/_convenience.py:105: FutureWarning: The default XML parser will be changing from 'xmlschema' to 'lxml' in version 0.4.0.  To silence this warning, please provide the `parser` argument, specifying either 'lxml' (to opt into the new behavior), or'xmlschema' (to retain the old behavior).
  d = to_dict(os.fspath(xml), parser=parser, validate=validate)
/opt/anaconda3/envs/napariNEW/lib/python3.9/site-packages/ome_types/_convenience.py:105: FutureWarning: The default XML parser will be changing from 'xmlschema' to 'lxml' in version 0.4.0.  To silence this warning, please provide the `parser` argument, specifying either 'lxml' (to opt into the new behavior), or'xmlschema' (to retain the old behavior).
  d = to_dict(os.fspath(xml), parser=parser, validate=validate)
/opt/anaconda3/envs/napariNEW/lib/python3.9/site-packages/ome_types/_convenience.py:105: FutureWarning: The default XML parser will be changing from 'xmlschema' to 

'/Users/ahenrie/Projects/Imaging/mcz_subcell/data/inferred_objects/SO_object3.ome.tiff'

In [9]:
###################
# PRE_PROCESSING
###################

#NU_eroded1 = morphology.binary_erosion(NU_object,  footprint=morphology.ball(3) )
NU_eroded = morphology.binary_erosion(NU_object)  
###################
# CORE_PROCESSING
###################

# logical  SO - NU_eroded
CY_object = np.logical_and(SO_object,~NU_eroded)


In [11]:

CY_object_filen = export_ome_tiff(CY_object, meta_dict_t, "CY_object", "./out_data/", curr_chan=0)
#CY_signal_filen = export_ome_tiff(CY_signal, meta_in, "CY_signal", "./out_data/", curr_chan=0)


viewer = napari.view_image( #viewer.add_image(
    CY_object,
    scale=scale,
    opacity=0.3,
)    


['CY_object']


/opt/anaconda3/envs/napariNEW/lib/python3.9/site-packages/ome_types/_convenience.py:105: FutureWarning: The default XML parser will be changing from 'xmlschema' to 'lxml' in version 0.4.0.  To silence this warning, please provide the `parser` argument, specifying either 'lxml' (to opt into the new behavior), or'xmlschema' (to retain the old behavior).
  d = to_dict(os.fspath(xml), parser=parser, validate=validate)


FileNotFoundError: [Errno 2] No such file or directory: '/Users/ahenrie/Projects/Imaging/infer-subc/notebooks/./out_data/CY_object.ome.tiff'

# DEFINE `infer_CYTOSOL` function

In [ ]:
# copy this to base.py for easy import

def infer_CYTOSOL(struct_img, in_params) -> tuple:
    """
    Procedure to infer NUCLEI from linearly unmixed input.

    Parameters:
    ------------
    struct_img: np.ndarray
        a 3d image containing the NUCLEI signal

    in_params: dict
        holds the needed parameters

    Returns:
    -------------
    tuple of:
        object
            mask defined boundaries of NU
        label
            label (could be more than 1)
        signal
            scaled/filtered (pre-processed) flourescence image
        parameters: dict
            updated parameters in case any needed were missing
    
    """

    out_p= in_params.copy()


    ###################
    # PRE_PROCESSING
    ###################                         

    #TODO: replace params below with the input params
    scaling_param =  [0]   
    struct_img = intensity_normalization(struct_img, scaling_param=scaling_param)
    out_p["intensity_norm_param"] = scaling_param

    med_filter_size = 4   
    # structure_img_median_3D = ndi.median_filter(struct_img,    size=med_filter_size  )
    struct_img = median_filter_slice_by_slice( 
                                                                    struct_img,
                                                                    size=med_filter_size  )
    out_p["median_filter_size"] = med_filter_size 

    gaussian_smoothing_sigma = 1.34
    gaussian_smoothing_truncate_range = 3.0
    struct_img = image_smoothing_gaussian_slice_by_slice(   struct_img,
                                                                                                        sigma=gaussian_smoothing_sigma,
                                                                                                        truncate_range = gaussian_smoothing_truncate_range
                                                                                                    )
    out_p["gaussian_smoothing_sigma"] = gaussian_smoothing_sigma 
    out_p["gaussian_smoothing_truncate_range"] = gaussian_smoothing_truncate_range

    ###################
    # CORE_PROCESSING
    ###################

    struct_obj = struct_img > filters.threshold_li(struct_img)
    threshold_value_log = threshold_li_log(struct_img)

    threshold_factor = 0.9 #from cellProfiler
    thresh_min = .1
    thresh_max = 1.
    threshold = min( max(threshold_value_log*threshold_factor, thresh_min), thresh_max)
    out_p['threshold_factor'] = threshold_factor
    out_p['thresh_min'] = thresh_min
    out_p['thresh_max'] = thresh_max

    struct_obj = struct_img > threshold

    ###################
    # POST_PROCESSING
    ###################

    hole_width = 5  
    # # wrapper to remoce_small_objects
    struct_obj = morphology.remove_small_holes(struct_obj, hole_width ** 3 )
    out_p['hole_width'] = hole_width


    small_object_max = 5
    struct_obj = aicssegmentation.core.utils.size_filter(struct_obj, # wrapper to remove_small_objects which can do slice by slice
                                                            min_size= small_object_max**3, 
                                                         method = "slice_by_slice", #"3D", # 
                                                            connectivity=1)
    out_p['small_object_max'] = small_object_max


    retval = (struct_obj,  label(struct_obj), NU_signal, out_p)
    return retval


In [ ]:
CY_object_filen = export_ome_tiff(SO_object, meta_dict, object_name, str(out_path)+"/", curr_chan=0)
